<a href="https://colab.research.google.com/github/tselane2110/Explainable-AI-for-Robot-Telemetry-Data/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Important Note: Since my roll number is `even`, therefore I will be training 1-D CNN, XGBoost and FNN only.

### 0. Importing Libraries

In [1]:
# importing required libraries
import pandas as pd
import numpy as np
import os
import pandas as pd
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import time
from tensorflow.keras.utils import to_categorical

### 1. Cloning the github repo to extract the dataset

In [2]:
!git clone https://github.com/tselane2110/Explainable-AI-for-Robot-Telemetry-Data

Cloning into 'Explainable-AI-for-Robot-Telemetry-Data'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 15 (delta 4), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (15/15), 2.04 MiB | 31.63 MiB/s, done.
Resolving deltas: 100% (4/4), done.


### 2. Extracting the Dataset

In [3]:
!unzip /content/Explainable-AI-for-Robot-Telemetry-Data/RawData -d /content/

Archive:  /content/Explainable-AI-for-Robot-Telemetry-Data/RawData.zip
   creating: /content/RawData/
   creating: /content/RawData/Dos-Drone/
  inflating: /content/RawData/Dos-Drone/Dos1.csv  
  inflating: /content/RawData/Dos-Drone/Dos2.csv  
   creating: /content/RawData/Malfunction-Drone/
  inflating: /content/RawData/Malfunction-Drone/Malfunction1.csv  
  inflating: /content/RawData/Malfunction-Drone/Malfunction2.csv  
   creating: /content/RawData/NormalFlight/
  inflating: /content/RawData/NormalFlight/Normal1.csv  
  inflating: /content/RawData/NormalFlight/Normal2.csv  
  inflating: /content/RawData/NormalFlight/Normal3.csv  
  inflating: /content/RawData/NormalFlight/Normal4.csv  
  inflating: /content/RawData/ReadMeForDataSet.txt  


### 3. Loading Datasets

In [4]:
# reading the csv files

df_dos1 = pd.read_csv("/content/RawData/Dos-Drone/Dos1.csv")
df_dos2 = pd.read_csv("/content/RawData/Dos-Drone/Dos2.csv")
df_malfunction1 = pd.read_csv("/content/RawData/Malfunction-Drone/Malfunction1.csv")
df_malfunction2 = pd.read_csv("/content/RawData/Malfunction-Drone/Malfunction2.csv")
df_normal1 = pd.read_csv("/content/RawData/NormalFlight/Normal1.csv")
df_normal2 = pd.read_csv("/content/RawData/NormalFlight/Normal2.csv")
df_normal3 = pd.read_csv("/content/RawData/NormalFlight/Normal3.csv")
df_normal4 = pd.read_csv("/content/RawData/NormalFlight/Normal4.csv")

#### DoS Datasets

In [5]:
df_dos1.head(5)

,S.No,setpoint_raw-global_Time,setpoint_raw-global_header.seq,setpoint_raw-global_header.stamp.secs,setpoint_raw-global_latitude,setpoint_raw-global_longitude,setpoint_raw-global_altitude,battery_Time,battery_header.seq,battery_header.stamp.secs,...,state_guided,state_manual_input,state_system_status,RSSI_Time,RSSI_Quality,RSSI_Signal,CPU_Time,CPU_Percent,RAM_Time,Used_RAM_MB
0,0,1.729405e+09,1,0,33.69785,73.016496,1.25,1.729405e+09,4605.0,1.729405e+09,...,1.0,1.0,5.0,1.729405e+09,1.0,-39.0,1.729405e+09,15.2,1.729405e+09,5.6
1,1,1.729405e+09,2,0,33.69785,73.016496,1.25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1.729405e+09,3,0,33.69785,73.016496,1.25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1.729405e+09,4,0,33.69785,73.016496,1.25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1.729405e+09,5,0,33.69785,73.016496,1.25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df_dos2.head(5)

,S.No,setpoint_raw-global_Time,setpoint_raw-global_header.seq,setpoint_raw-global_header.stamp.secs,setpoint_raw-global_latitude,setpoint_raw-global_longitude,setpoint_raw-global_altitude,battery_Time,battery_header.seq,battery_header.stamp.secs,...,state_guided,state_manual_input,state_system_status,RSSI_Time,RSSI_Quality,RSSI_Signal,CPU_Time,CPU_Percent,RAM_Time,Used_RAM_MB
0,0,1.729787e+09,570,0,33.636530,72.99146,1.279,1.729787e+09,12473.0,1.729787e+09,...,0.0,1.0,5.0,1.729787e+09,0.642857,-65.0,1.729787e+09,3.9,1.729787e+09,6.5
1,1,1.729787e+09,664,0,33.636530,72.99146,1.279,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1.729787e+09,1177,0,33.636530,72.99146,1.279,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1.729787e+09,1374,0,33.636529,72.99146,1.272,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1.729787e+09,1417,0,33.636529,72.99146,1.272,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# adding a new column in both dos1 and dos2
df_dos1["class"] = "DoS_Attack"
df_dos2["class"] = "DoS_Attack"

#### Malfunction Datasets

In [8]:
df_malfunction1.head(5)

,S.No,setpoint_raw-global_Time,setpoint_raw-global_header.seq,setpoint_raw-global_header.stamp.secs,setpoint_raw-global_latitude,setpoint_raw-global_longitude,setpoint_raw-global_altitude,battery_Time,battery_header.seq,battery_header.stamp.secs,...,state_guided,state_manual_input,state_system_status,RSSI_Time,RSSI_Quality,RSSI_Signal,CPU_Time,CPU_Percent,RAM_Time,Used_RAM_MB
0,0,1.729406e+09,1,0,33.69788,73.016504,1.25,1.729406e+09,8332.0,1.729406e+09,...,1.0,1.0,5.0,1.729406e+09,0.985714,-41.0,1.729406e+09,44.9,1.729406e+09,11.2
1,1,1.729406e+09,2,0,33.69788,73.016504,1.25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1.729406e+09,3,0,33.69788,73.016504,1.25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1.729406e+09,4,0,33.69788,73.016504,1.25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1.729406e+09,5,0,33.69788,73.016504,1.25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_malfunction2.head(5)

,S.No,setpoint_raw-global_Time,setpoint_raw-global_header.seq,setpoint_raw-global_header.stamp.secs,setpoint_raw-global_latitude,setpoint_raw-global_longitude,setpoint_raw-global_altitude,battery_Time,battery_header.seq,battery_header.stamp.secs,...,state_guided,state_manual_input,state_system_status,RSSI_Time,RSSI_Quality,RSSI_Signal,CPU_Time,CPU_Percent,RAM_Time,Used_RAM_MB
0,0,1.729688e+09,59,0,33.63709,72.991302,1.25,1.729688e+09,21351.0,1.729688e+09,...,0.0,1.0,5.0,1.729688e+09,0.8,-54.0,1.729688e+09,7.3,1.729688e+09,7.6
1,1,1.729688e+09,66,0,33.63709,72.991303,1.25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1.729688e+09,369,0,33.63709,72.991303,1.25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1.729688e+09,2111,0,33.63709,72.991303,1.25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1.729688e+09,2112,0,33.63709,72.991303,1.25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# adding a new column in both malfunction1 and malfunction2
df_malfunction1["class"] = "Malfunction"
df_malfunction2["class"] = "Malfunction"

#### Normal Datasets

In [11]:
df_normal1.head(5)

,S.No,setpoint_raw-global_Time,setpoint_raw-global_header.seq,setpoint_raw-global_header.stamp.secs,setpoint_raw-global_latitude,setpoint_raw-global_longitude,setpoint_raw-global_altitude,battery_Time,battery_header.seq,battery_header.stamp.secs,...,state_guided,state_manual_input,state_system_status,RSSI_Time,RSSI_Quality,RSSI_Signal,CPU_Time,CPU_Percent,RAM_Time,Used_RAM_MB
0,0,1.729315e+09,137,0,33.707308,73.019849,1.25,1.729315e+09,14916.0,1.729315e+09,...,0.0,1.0,5.0,1.729315e+09,1.0,-40.0,1.729315e+09,9.6,1.729315e+09,5.9
1,1,1.729315e+09,139,0,33.707308,73.019849,1.25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1.729315e+09,141,0,33.707308,73.019849,1.25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1.729315e+09,147,0,33.707308,73.019849,1.25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1.729315e+09,155,0,33.707308,73.019849,1.25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_normal2.head(5)

,S.No,setpoint_raw-global_Time,setpoint_raw-global_header.seq,setpoint_raw-global_header.stamp.secs,setpoint_raw-global_latitude,setpoint_raw-global_longitude,setpoint_raw-global_altitude,battery_Time,battery_header.seq,battery_header.stamp.secs,...,state_guided,state_manual_input,state_system_status,RSSI_Time,RSSI_Quality,RSSI_Signal,CPU_Time,CPU_Percent,RAM_Time,Used_RAM_MB
0,0,1.729686e+09,2,0,33.637079,72.991263,1.25,1.729686e+09,0.0,1.729686e+09,...,0.0,1.0,3.0,1.729686e+09,1.0,-40.0,1.729686e+09,0.8,1.729686e+09,5.6
1,1,1.729686e+09,3,0,33.637079,72.991263,1.25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1.729686e+09,4,0,33.637079,72.991263,1.25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1.729686e+09,5,0,33.637079,72.991263,1.25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1.729686e+09,6,0,33.637079,72.991263,1.25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df_normal3.head(5)

,S.No,setpoint_raw-global_Time,setpoint_raw-global_header.seq,setpoint_raw-global_header.stamp.secs,setpoint_raw-global_latitude,setpoint_raw-global_longitude,setpoint_raw-global_altitude,battery_Time,battery_header.seq,battery_header.stamp.secs,...,state_guided,state_manual_input,state_system_status,RSSI_Time,RSSI_Quality,RSSI_Signal,CPU_Time,CPU_Percent,RAM_Time,Used_RAM_MB
0,0,1.729874e+09,351,0,33.636576,72.991521,1.25,1.729874e+09,8469.0,1.729874e+09,...,0.0,1.0,5.0,1.729874e+09,0.957143,-43.0,1.729874e+09,6.2,1.729874e+09,5.5
1,1,1.729874e+09,355,0,33.636576,72.991521,1.25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1.729874e+09,1648,0,33.636576,72.991521,1.25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1.729874e+09,1699,0,33.636576,72.991521,1.25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1.729874e+09,1826,0,33.636576,72.991521,1.25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df_normal4.head(5)

,S.No,setpoint_raw-global_Time,setpoint_raw-global_header.seq,setpoint_raw-global_header.stamp.secs,setpoint_raw-global_latitude,setpoint_raw-global_longitude,setpoint_raw-global_altitude,battery_Time,battery_header.seq,battery_header.stamp.secs,...,state_guided,state_manual_input,state_system_status,RSSI_Time,RSSI_Quality,RSSI_Signal,CPU_Time,CPU_Percent,RAM_Time,Used_RAM_MB
0,0,1.729786e+09,7,0,33.636539,72.991440,1.25,1.729786e+09,270.0,1.729786e+09,...,0.0,1.0,3.0,1.729786e+09,0.814286,-53.0,1.729786e+09,1.3,1.729786e+09,5.7
1,1,1.729786e+09,452,0,33.636539,72.991440,1.25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1.729786e+09,1206,0,33.636539,72.991439,1.25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1.729786e+09,1235,0,33.636539,72.991439,1.25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1.729786e+09,1247,0,33.636539,72.991439,1.25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# adding a new column in all normal-drone dfs
df_normal1["class"] = "Normal"
df_normal2["class"] = "Normal"
df_normal3["class"] = "Normal"
df_normal4["class"] = "Normal"

### Concatenating all datasets and dropping duplicates

In [16]:
dfs = [df_dos1, df_dos2, df_malfunction1, df_malfunction2, df_normal1, df_normal2, df_normal3, df_normal4]

df = pd.concat(dfs, ignore_index=True).drop_duplicates()


### Data Preprocessing

#### Dropping Null rows and cols

In [17]:
df = df.dropna(axis=1, how='all').dropna(axis=0, how='all')

In [18]:
df.describe()

/usr/local/lib/python3.12/dist-packages/pandas/core/nanops.py:1016: RuntimeWarning: overflow encountered in square
  sqr = _ensure_numeric((avg - values) ** 2)
/usr/local/lib/python3.12/dist-packages/pandas/core/nanops.py:1016: RuntimeWarning: overflow encountered in square
  sqr = _ensure_numeric((avg - values) ** 2)


,S.No,setpoint_raw-global_Time,setpoint_raw-global_header.seq,setpoint_raw-global_header.stamp.secs,setpoint_raw-global_latitude,setpoint_raw-global_longitude,setpoint_raw-global_altitude,battery_Time,battery_header.seq,battery_header.stamp.secs,...,state_guided,state_manual_input,state_system_status,RSSI_Time,RSSI_Quality,RSSI_Signal,CPU_Time,CPU_Percent,RAM_Time,Used_RAM_MB
count,87417.000000,8.741700e+04,87417.000000,87417.0,87417.000000,87417.000000,87417.000000,4.349000e+03,4349.000000,4.349000e+03,...,466.000000,466.0,466.000000,8.500000e+01,85.000000,85.000000,1.820000e+02,182.000000,5.530000e+02,553.000000
mean,6633.756363,1.729593e+09,126929.651487,0.0,33.664535,73.002702,4.374865,1.729626e+09,8946.337089,1.729626e+09,...,0.699571,1.0,4.841202,1.729613e+09,0.902185,-46.470588,1.729647e+09,15.958791,1.729644e+09,10.234720
std,4566.256233,2.001018e+05,98296.621753,0.0,0.031810,0.012993,2.820141,2.045361e+05,6819.779788,2.045361e+05,...,0.458937,0.0,0.512739,2.072144e+05,0.095030,7.105645,1.975626e+05,15.360037,1.964418e+05,10.221969
min,0.000000,1.729315e+09,1.000000,0.0,33.636529,72.991018,1.250000,1.729315e+09,0.000000,1.729315e+09,...,0.000000,1.0,3.000000,1.729315e+09,0.628571,-66.000000,1.729315e+09,0.000000,1.729315e+09,4.900000
25%,2781.000000,1.729405e+09,52717.000000,0.0,33.636642,72.991309,2.269000,1.729405e+09,712.000000,1.729405e+09,...,0.000000,1.0,5.000000,1.729405e+09,0.828571,-52.000000,1.729406e+09,7.525000,1.729406e+09,5.600000
50%,5903.000000,1.729686e+09,118526.000000,0.0,33.637122,72.991634,3.451000,1.729686e+09,8704.000000,1.729686e+09,...,1.000000,1.0,5.000000,1.729688e+09,0.914286,-46.000000,1.729688e+09,10.500000,1.729688e+09,5.800000
75%,9794.000000,1.729786e+09,176584.000000,0.0,33.697880,73.016555,5.978000,1.729787e+09,15015.000000,1.729787e+09,...,1.000000,1.0,5.000000,1.729786e+09,1.000000,-40.000000,1.729787e+09,16.725000,1.729787e+09,6.600000
max,17868.000000,1.729874e+09,450486.000000,0.0,33.707439,73.019869,15.985000,1.729874e+09,21832.000000,1.729874e+09,...,1.000000,1.0,5.000000,1.729874e+09,1.000000,-34.000000,1.729874e+09,99.300000,1.729874e+09,44.000000


In [19]:
# dropping S.NO bcz not needed
df.drop(columns=["S.No"], inplace=True)

In [20]:
df.columns, len(df.columns)

(Index(['setpoint_raw-global_Time', 'setpoint_raw-global_header.seq',
        'setpoint_raw-global_header.stamp.secs', 'setpoint_raw-global_latitude',
        'setpoint_raw-global_longitude', 'setpoint_raw-global_altitude',
        'battery_Time', 'battery_header.seq', 'battery_header.stamp.secs',
        'battery_voltage', 'battery_current', 'battery_temperature',
        'battery_percentage', 'global_position-local_Time',
        'global_position-local_header.seq',
        'global_position-local_header.stamp.secs',
        'global_position-local_pose.pose.position.x',
        'global_position-local_pose.pose.position.y',
        'global_position-local_pose.pose.position.z',
        'global_position-local_pose.pose.orientation.x',
        'global_position-local_pose.pose.orientation.y',
        'global_position-local_pose.pose.orientation.z',
        'global_position-local_twist.twist.linear.x',
        'global_position-local_twist.twist.linear.y',
        'global_position-local_twist

In [21]:
# how many null rows each column has
null_stats = pd.DataFrame({
    "null_count": df.isna().sum(),
    "null_percent": df.isna().mean() * 100
})

print(null_stats.to_string())

                                               null_count  null_percent
setpoint_raw-global_Time                                0      0.000000
setpoint_raw-global_header.seq                          0      0.000000
setpoint_raw-global_header.stamp.secs                   0      0.000000
setpoint_raw-global_latitude                            0      0.000000
setpoint_raw-global_longitude                           0      0.000000
setpoint_raw-global_altitude                            0      0.000000
battery_Time                                        83068     95.024995
battery_header.seq                                  83068     95.024995
battery_header.stamp.secs                           83068     95.024995
battery_voltage                                     83068     95.024995
battery_current                                     83068     95.024995
battery_temperature                                 83068     95.024995
battery_percentage                                  83068     95

This is bad!  But we probably can't delete all these so will look for a solution!

### Handling Missing Data

In [22]:
# Step 1: Drop columns with more than 80% null values
null_percent = df.isna().mean() * 100
columns_to_keep = null_percent[null_percent <= 80].index.tolist()
df = df[columns_to_keep].copy()

# Step 2: Keep only rows that have at least 3 non-null values
df = df[df.notna().sum(axis=1) >= 3].copy()

# Step 3: Fill remaining nulls with median for numeric, mode for categorical
for col in df.columns:
    if df[col].dtype in ['float64', 'int64']:
        df[col] = df[col].fillna(df[col].median())
    else:
        if col != 'class':
            df[col] = df[col].fillna(df[col].mode()[0] if not df[col].mode().empty else 0)

# Step 4: Encode the target variable
df['class_encoded'] = df['class'].map({'Normal': 0, 'DoS_Attack': 1, 'Malfunction': 2})

# Step 5: Split features and target
X = df.drop(['class', 'class_encoded'], axis=1)
y = df['class_encoded']

# Step 6: Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 7: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

print(f"Cleaned data shape: {df.shape}")
print(f"Training samples: {X_train.shape[0]}")
print(f"Testing samples: {X_test.shape[0]}")
print(f"Features: {X_train.shape[1]}")

Cleaned data shape: (87417, 8)
Training samples: 69933
Testing samples: 17484
Features: 6


### 1. XGBoost

In [23]:
# Step 8: Train XGBoost (quick first model)
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    random_state=42,
    eval_metric='mlogloss'
)
xgb_model.fit(X_train, y_train)

# Step 9: Predict and evaluate
y_pred = xgb_model.predict(X_test)
print("XGBoost Results:")
print(classification_report(y_test, y_pred, target_names=['Normal', 'DoS_Attack', 'Malfunction']))

XGBoost Results:
              precision    recall  f1-score   support

      Normal       1.00      1.00      1.00      9960
  DoS_Attack       1.00      1.00      1.00      3918
 Malfunction       1.00      1.00      1.00      3606

    accuracy                           1.00     17484
   macro avg       1.00      1.00      1.00     17484
weighted avg       1.00      1.00      1.00     17484



### 2. 1-D CNN

In [24]:
# Step 10: Prepare data for 1D-CNN (reshape for time-series like structure)
# We'll treat each row as a "time step" with multiple features
X_train_cnn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_cnn = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Step 11: One-hot encode labels for CNN
y_train_cnn = to_categorical(y_train, num_classes=3)
y_test_cnn = to_categorical(y_test, num_classes=3)

# Step 12: Build 1D-CNN
cnn_model = keras.Sequential([
    layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train_cnn.shape[1], 1)),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(50, activation='relu'),
    layers.Dense(3, activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train_cnn, y_train_cnn, epochs=10, batch_size=32, validation_split=0.1, verbose=1)

# Step 13: Evaluate CNN
cnn_pred = cnn_model.predict(X_test_cnn)
cnn_pred_classes = np.argmax(cnn_pred, axis=1)
print("\n1D-CNN Results:")
print(classification_report(y_test, cnn_pred_classes, target_names=['Normal', 'DoS_Attack', 'Malfunction']))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1967/1967 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.8164 - loss: 0.4341 - val_accuracy: 0.9580 - val_loss: 0.1059
Epoch 2/10
1967/1967 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9669 - loss: 0.0971 - val_accuracy: 0.9750 - val_loss: 0.0757
Epoch 3/10
1967/1967 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9696 - loss: 0.0814 - val_accuracy: 0.9723 - val_loss: 0.0717
Epoch 4/10
1967/1967 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9705 - loss: 0.0741 - val_accuracy: 0.9733 - val_loss: 0.0830
Epoch 5/10
1967/1967 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9743 - loss: 0.0664 - val_accuracy: 0.9718 - val_loss: 0.0583
Epoch 6/10
1967/1967 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9728 - loss: 0.0630 - val_accuracy: 0.9745 - val_loss: 0.0571
Epoch 7/10
1967/1967 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9764 - loss: 0.0571 - val_accuracy: 0.9801 - val_loss: 0.0497
Epoch 8/10
1967/1967 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9769 - loss: 0.0541 

### 3. FNN

In [25]:
# Step 14: Build FNN (Feed Forward Neural Network)
fnn_model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(32, activation='relu'),
    layers.Dense(3, activation='softmax')
])

fnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
fnn_model.fit(X_train, y_train, epochs=15, batch_size=32, validation_split=0.1, verbose=1)

# Step 15: Evaluate FNN
fnn_pred = fnn_model.predict(X_test)
fnn_pred_classes = np.argmax(fnn_pred, axis=1)
print("\nFNN Results:")
print(classification_report(y_test, fnn_pred_classes, target_names=['Normal', 'DoS_Attack', 'Malfunction']))

Epoch 1/15


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1967/1967 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.8203 - loss: 0.4126 - val_accuracy: 0.9487 - val_loss: 0.0907
Epoch 2/15
1967/1967 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9610 - loss: 0.0958 - val_accuracy: 0.9733 - val_loss: 0.0908
Epoch 3/15
1967/1967 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9699 - loss: 0.0743 - val_accuracy: 0.9814 - val_loss: 0.0464
Epoch 4/15
1967/1967 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9723 - loss: 0.0663 - val_accuracy: 0.9748 - val_loss: 0.0591
Epoch 5/15
1967/1967 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9761 - loss: 0.0596 - val_accuracy: 0.9804 - val_loss: 0.0482
Epoch 6/15
1967/1967 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9754 - loss: 0.0593 - val_accuracy: 0.9811 - val_loss: 0.0429
Epoch 7/15
1967/1967 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9759 - loss: 0.0596 - val_accuracy: 0.9757 - val_loss: 0.0492
Epoch 8/15
1967/1967 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9751 - loss: 0.0586 - val_accura

In [26]:
# Step 16: Compare all models
print("Model Comparison:")
print("-" * 50)
print(f"XGBoost Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"1D-CNN Accuracy: {accuracy_score(y_test, cnn_pred_classes):.4f}")
print(f"FNN Accuracy: {accuracy_score(y_test, fnn_pred_classes):.4f}")

Model Comparison:
--------------------------------------------------
XGBoost Accuracy: 1.0000
1D-CNN Accuracy: 0.9798
FNN Accuracy: 0.9723
